In [462]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline

In [382]:
!pip install category_encoders
import category_encoders as ce

In [383]:
pd.options.mode.chained_assignment = None

In [384]:
df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vTMIUP6_JIoxWSAFCe1h6Hz12r-41t6qHv5cCXIBmYJUK2KS188pKkZnkr4jJRpIcC3mRZV36z21oNv/pub?gid=0&single=true&output=csv')

In [385]:
header_list = ['Date','Weight','Fat','Sleep Debt','REM','Deep Sleep','Snore',
     'Meditate','Spanish','Push-ups','Pull-ups','Sit-ups','Coffee','Handstands',
     'Acro','Swing','Strain','Calories','AHR','MHR','HRV','RHR','Recovery',
     'Carbs','Journal','Spinal Mobility','Flexibility','Notes','DOW',
     'Weight_AVR','Fat_AVR','Sleep Debt_AVR','REM_AVR','Deep Sleep_AVR',
     'Strain_AVR','Calories_AVR','AHR_AVR','MHR_AVR','HRV_AVR','RHR_AVR',
     'Recovery_AVR','Weight_PASS','Fat_PASS','Sleep Debt_PASS','REM_PASS',
     'Deep Sleep_PASS','Strain_PASS','Calories_PASS','AHR_PASS','MHR_PASS',
     'HRV_PASS','RHR_PASS','Recovery_PASS']

In [386]:
df = df.reindex(columns = header_list)

In [387]:
df['Date'] = pd.to_datetime(df['Date'])

for i in range(len(df)):
    df['DOW'][i] = df['Date'][i].day_name()

In [388]:
features = ['Weight','Fat', 'Sleep Debt', 'REM','Deep Sleep','Strain','Calories','AHR','MHR','HRV','RHR','Recovery']
special_features = ['Sleep Debt','AHR','RHR']
drops = ['Acro','Meditate','Snore','Coffee','Handstands','Spanish','Push-ups',
         'Pull-ups','Sit-ups','Swing','Carbs','Journal','Spinal Mobility',
         'Flexibility','Notes']
week_day = ['Sunday','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday']

In [389]:
for each in features:
    df[each] = df[each].interpolate(method='linear')

In [390]:
df = df.drop(labels=drops,axis=1)

In [391]:
for i in range(0,df.shape[0]-1):
    for j in range(len(features)):
        string = features[j] + '_AVR'
        df[string] = df.iloc[:,j+1].expanding(min_periods=7).mean()

In [392]:
for i in range(0,df.shape[0]-1):
    for j in range(len(features)):
            score = features[j] + '_PASS'
            avr = features[j] + '_AVR'
            if df[features[j]][i] >= df[avr][i]:
                df[score][i] = "Y"
            else:
                df[score][i] = "N"


In [393]:
for i in range(0,df.shape[0]-1):
    for j in range(len(special_features)):
            score = special_features[j] + '_PASS'
            if df[special_features[j]][i] <= df[avr][i]:
                df[score][i] = "Y"
            else:
                df[score][i] = "N"

Storing some code for rolling average & exponential average here.

    df['Rolling_AVR'] = df.iloc[:,1].rolling(window=7).mean()
    
    

In [394]:
df.describe()

,Weight,Fat,Sleep Debt,REM,Deep Sleep,Strain,Calories,AHR,MHR,HRV,...,Sleep Debt_AVR,REM_AVR,Deep Sleep_AVR,Strain_AVR,Calories_AVR,AHR_AVR,MHR_AVR,HRV_AVR,RHR_AVR,Recovery_AVR
count,587.000000,587.000000,587.000000,587.000000,587.000000,587.000000,587.000000,587.000000,587.000000,587.000000,...,581.000000,581.000000,581.000000,581.000000,581.000000,581.000000,581.000000,581.000000,581.000000,581.000000
mean,152.286882,13.389949,86.613288,74.321976,73.092845,11.532112,2808.166099,79.316014,158.930153,33.931857,...,94.023714,51.161490,66.599461,11.833818,3121.237098,78.537071,156.776707,32.092449,65.476934,49.909536
std,3.429221,0.790793,24.991448,34.916713,26.893715,2.074627,796.823336,3.957058,6.840874,8.322013,...,6.731922,17.817770,5.898965,0.341223,154.483685,0.429255,1.811351,2.363854,0.525731,1.257957
min,146.800000,12.200000,0.000000,2.000000,2.000000,4.600000,449.000000,71.000000,132.000000,10.000000,...,81.081633,15.164286,59.182692,10.997115,2808.166099,77.744709,154.131868,24.204082,64.807065,47.496374
25%,148.932692,12.562019,74.115385,49.427885,56.235577,10.348810,2153.500000,77.103632,154.988095,28.657143,...,89.144501,37.893725,62.640741,11.594821,2992.698936,78.222457,154.800816,32.243772,65.059955,48.900192
50%,151.400000,13.200000,88.230769,71.000000,74.244444,11.894231,2865.961538,78.230769,157.000000,35.000000,...,90.464724,53.545455,64.324257,11.810185,3164.770042,78.484456,157.153199,32.791068,65.361905,50.327830
75%,155.821795,14.182692,101.076923,98.055556,86.310897,12.936859,3346.500000,81.000000,162.000000,37.803846,...,98.481308,66.981900,69.481250,12.154068,3267.936813,78.925697,158.465909,33.671141,65.807416,50.665865
max,159.600000,15.000000,192.000000,200.000000,165.000000,16.800000,5660.000000,93.000000,193.000000,81.000000,...,107.643300,74.321976,84.927778,12.310491,3346.306122,79.323932,159.300195,33.970986,66.627205,53.413265


In [395]:
test = df[df['Date'] >= '12/03/2019']
train = df[df['Date'] <= '08/06/2019']
val = df[(df['Date'] <= '12/02/2019') & (df['Date'] >= '08/07/2019')]

In [396]:
target = 'Sleep Debt_PASS'
# Get a dataframe with all train columns except the target
train_features = train.drop(columns=[target])

# Get a list of the numeric features
numeric_features = train_features.select_dtypes(include='number').columns.tolist()

# Get a series with the cardinality of the nonnumeric features
cardinality = train_features.select_dtypes(exclude='number').nunique()

# Get a list of all categorical features with cardinality <= 50
categorical_features = cardinality[cardinality <= 25].index.tolist()

# Combine the lists 
features = numeric_features + categorical_features

In [397]:
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

In [398]:
pipeline = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True), 
    SimpleImputer(strategy='mean'), 
    RandomForestClassifier(max_features='log2',n_estimators=100,n_jobs=-1, random_state=42)
)

In [399]:
pipeline.fit(X_train, y_train)
print('Train Accuracy', pipeline.score(X_train, y_train))
print('Validation Accuracy', pipeline.score(X_val, y_val))

Train Accuracy 1.0
Validation Accuracy 0.9491525423728814


In [400]:
y_pred = pipeline.predict(X_test)

In [401]:
print('Test Accuracy', pipeline.score(X_test, y_pred))

Test Accuracy 1.0


In [475]:
df['Sleep Debt'][538], df['Sleep Debt_PASS'][538], test['Sleep Debt'][538], X_test['Sleep Debt'][538], np.array(y_pred)[10],X_test['Sleep Debt_AVR'][538]

(40.0, 'Y', 40.0, 40.0, 'N', 88.21521335807049)

In [476]:
df['Sleep Debt'][537], df['Sleep Debt_PASS'][537], test['Sleep Debt'][537], X_test['Sleep Debt'][537], np.array(y_pred)[9],X_test['Sleep Debt_AVR'][537]

(57.0, 'N', 57.0, 57.0, 'N', 88.30483271375464)

In [477]:
df['Sleep Debt'][536], df['Sleep Debt_PASS'][536], test['Sleep Debt'][536], X_test['Sleep Debt'][536], np.array(y_pred)[8], X_test['Sleep Debt_AVR'][536]

(55.0, 'N', 55.0, 55.0, 'N', 88.3631284916201)

for each in week_day:
    frame = df[df['DOW'] == each]
    for i in features:
        print(each, i,frame[i].mean())

In [ ]:
for i in features:
    print(i,df[i].mean())